In [ ]:
!pip install pytube
!pip install pydub
!pip install TTS
from pydub import AudioSegment
from pytube import YouTube
import IPython

In [ ]:
# Handling The Model.  Change As Needed.
import torch
from TTS.api import TTS
# Get device
device = "cuda" if torch.cuda.is_available() else "cpu"
# Init TTS
tts = TTS("tts_models/multilingual/multi-dataset/xtts_v2").to(device)

In [ ]:
# url of video that has the audio.
yt = YouTube("")
# Default to the first audio file on the server.  It is not always the highest quality, but on a good connection it should be.
# read the docs of pytube for more on how this can be changed. https://pytube.io/en/latest/
RawMP3 = yt.streams.filter(only_audio=True).first()
HoldRaw = RawMP3.download()

In [ ]:
# From here, it's just a bit of pydub and an array of time codes we are going to seek to and slice samples from.
# These samples could be saved to disk, but since everything on colab gets wiped at the end of the session, it's so
# much easier to hold them in memory.  We won't need them later, anyway.  Same thing with the time codes.  They could
# totally be pulled from a text file and read into an array.  But that's more overhead that we don't need.

TimeCodes = [] # IN MILLISECONDS!  It's going to be called as an int, so decimals will fuck it right the fuck up.

MashUp = AudioSegment.empty()

# Assuming there's at least one time range in the array to splice, we run a for loop through the array appending each splice
# into the AudioSegment.  It's probably best to also have a slight pause, just so the TTS can seperate phonems.  It's not strictly
# necessary, but TTS is weird.
for i in range(0,len(TimeCodes),2):
  MashUp = MashUp.append(int(TimeCodes[i]):int(TimeCodes[i+1]))
  MashUp = MashUp.appent(AudioSegment.silent(duration=500))
# It should be as close to 20 seconds as possible, and a half second (500 milliseconds) pause after each loop.  Obviously, the duration can be changed
# as needed.  The voice clone engine needs it to be a WAV, so it gets exported locally here.  Just to make the engine happy.  It SHOULD be able
# to just pull it from memory, but I don't want to fight it.
MashUp.export("ClipSample.wav", format="wav")

In [ ]:
# Use any Phenome Pangram here.  It's easy to find them online.  I'll put a few in as comments, just for the cause.
# It's worth noting tho, that some produce british accents.  The easiest way to undo that is use a different one.
# I think it has something to do with how recursive it is in cloning, but I don't care enought to check this theory.
#
# https://clagnut.com/blog/2380/#English_phonetic_pangrams
# "With tenure, Suzie’d have all the more leisure for yachting, but her publications are no good."
# "Shaw, those twelve beige hooks are joined if I patch a young, gooey mouth."
# "The beige hue on the waters of the loch impressed all, including the French queen, before she heard that symphony again, just as young Arthur wanted."
script="The beige hue on the waters of the loch impressed all, including the French queen, before she heard that symphony again, just as young Arthur wanted."
NewVoice = tts.tts(text=script, speaker_wav="ClipSample.wav", language="en")
# And now we save it as a WAV, since that's what the engine will want to pull from when it's used in the future.  But it WILL OVERWRITE ANYTHING
# THAT IS ALREADY THERE WITH THE SAME NAME.  SO IF YOU HAVE ONE YOU LIKE, MOVE IT OR SOMETHING.  PYDUB DOESN'T FUCK AROUND.
NewVoice.export("MasterCopy.wav", format="wav")
#
# IPython will smash in a little player, so you can preview it without downloading.  If it sounds good, you're done.  Now you can clone any speech
# off it.  The better it sounds here, the better it will clone.  Any problems it has here can be corrected by getting better samples.
# It should be very obvious how easy this makes ANY voice to reproduce.  Without retraining a whole model on a whole dataset.
IPython.display.Audio("MasterCopy.wav")